In [6]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

first_year = 1985
last_year = 2020

from utils import export_image
from utils import map_image


In [7]:
# import ages from MapBiomas
age = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v1').select('secondary_vegetation_age_2020')
# region of interest is the entire country
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()
# select ecoregions to run analysis
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi) \
                .map(lambda feature: feature.intersection(roi))
# biomass for 2020 comes from CCI Biomass
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass")
proj = biomass.projection().getInfo()
crs = proj['crs']
crsTransform = proj['transform']

In [8]:
# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
  .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte()
lulc_masked = lulc.updateMask(age)

# Remove secondary forests with undesired histories (explain this more)
LU_index_ee = ee.List([5,49,10,11,12,32,29,50,13,22,23,24,30,25,26,33,31,27])

# Make a mask with all pixels with undesired land use types
undesired_mask = lulc_masked.remap(LU_index_ee, ee.List.repeat(1, LU_index_ee.size()), 0)
# Reduce to a single raster mask which contains every 
undesired_mask = undesired_mask.reduce(ee.Reducer.allNonZero())
desired_mask = undesired_mask.Not()

lulc_masked = lulc_masked.updateMask(desired_mask)
age = age.updateMask(desired_mask)

fire = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1") \
  .select([f"burned_coverage_{year}" for year in range(first_year, last_year)]).byte()
fire = fire.updateMask(age)

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [9]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires').byte()

# how many years ago was each fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire').byte()

In [22]:
vis_params = {
    'min': 0,
    'max': 40,
    'palette': ['blue', 'green', 'red']
}

# get fire frequency data from Mapbiomas - double check it.
fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
fire_freq = fire_freq.select('fire_frequency_1985_2020').clip(ecoregions.first())

num_fires2 = num_fires.clip(ecoregions.first())

# Create a map centered at (lat, lon).
Map = geemap.Map()
# Add the fire frequency layer to the map.
Map.addLayer(fire_freq, vis_params, 'Fire Frequency')
# Add the number of fires layer to the map.
Map.addLayer(num_fires2, vis_params, 'Number of Fires')
# Display the map.
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
img_export = age.addBands([last_fire, num_fires])

# Specify the destination asset ID.
assetId = 'projects/ee-ana-zonia/assets/age_fire'

# Create an export task.
task = ee.batch.Export.image.toAsset(
    image=img_export,
    description='Export to GEE',
    assetId=assetId,
    scale=30,
    maxPixels=1e13
)

# Start the task.
task.start()



In [8]:
# vis = {
#     'min': 0,
#     'max': 34,
#     'palette': ['blue', 'red'],
# }

# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(num_fires, vis, 'num_fires')
# Map.addLayer(age, vis, 'age')
# Map.addLayer(LU_sum.select('lulc_sum_15'), vis, 'LU_sum')
# Map.addLayer(last_fire, vis, 'last_fire')
# Map

### Total sum of land use types

In [25]:
LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9]

LU_sum = ee.Image()

for val in LU_index:
  lulc_val = lulc_masked.eq(val)
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

LU_sum = LU_sum.slice(1).byte()

In [29]:
img_export = age.addBands([LU_sum, last_fire, num_fires])

task = ee.batch.Export.image.toAsset(
    image=last_LU,
    description='last_LU',
    assetId='projects/ee-ana-zonia/assets/last_LU',
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e10
)

# Start the export task
task.start()

### Last Land Use Type

In [28]:
years = range(1986, 2020)

last_LU = ee.Image()

for yr in years:
    year = f'classification_{yr}'
    lu_yr = lulc_masked.select(year)
    age_mask = age.eq(2020 - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask).updateMask(lu_yr.neq(3)).updateMask(lu_yr.neq(6)))

last_LU = last_LU.slice(1)

# Convert ImageCollection to single Image
last_LU = last_LU.reduce(ee.Reducer.sum()).rename('last_LU')

### Fallow period length

In [30]:
nat_cover = lulc_masked.updateMask((lulc.neq(4)).neq(5)).lt(7)
nat_cover = nat_cover.updateMask(nat_cover).reduce(ee.Reducer.sum())
fallow = nat_cover.subtract(age).subtract(1).rename('fallow')

task = ee.batch.Export.image.toAsset(
    image=fallow,
    description='fallow',
    assetId='projects/ee-ana-zonia/assets/fallow',
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e10
)

# Start the export task
task.start()

## Mature Forests

In [31]:
# Mask the image to keep only pixels with natural vegetation cover in all bands
# 1 = forest
# 3 = forest
# 4 = savanna
# 5 = mangrove
# 6 = flooded forest
nat_cover = lulc.updateMask((lulc.neq(4)).neq(5)).lt(7)
mature_mask = nat_cover.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask)

# Compute the mean biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer = ee.Reducer.median(),\
                                             scale = 10000, crs = crs)

# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['median'], ee.Reducer.first())

task = ee.batch.Export.image.toAsset(
    image=median_mature,
    description='median_mature',
    assetId='projects/ee-ana-zonia/assets/median_mature',
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e10
)

# Start the export task
task.start()

## Export

In [9]:
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
land_use = LU_sum.addBands([last_LU, last_fire, num_fires, frag, median_mature])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si")
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec")
cwd_chave = ee.Image("projects/ee-ana-zonia/assets/cwd_chave").clip(roi)
climate = yearly_si.addBands([mean_prec, cwd_chave])

img_export = age.addBands([biomass, land_use, climate, categorical, age.pixelLonLat()]).updateMask(age)


In [13]:
fishnet = geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=0.5)
# Set 'id' as a property of the features
fishnet = fishnet.map(lambda feature: feature.set('id', feature.id()))
fishnet_info = fishnet.getInfo()['features']

In [14]:
for feature in fishnet_info:
    id = feature['id']
    grid_cell = fishnet.filter(ee.Filter.eq('id', id))
    img = img_export.clip(grid_cell)
    task = ee.batch.Export.image.toAsset(
        image = img_export,
        description = 'unified_data',
        assetId = f'projects/ee-ana-zonia/assets/unified_data_{id}',
        scale = 30,
        crs = 'EPSG:4326',
        maxPixels = 4e12
    )
    # Start the export task
    task.start()

In [ ]:
fc_sampled = img_export.stratifiedSample(10000, 'ecoreg')

task = ee.batch.Export.table.toDrive(
    collection = fc_sampled,
    description = 'campo_dataset',
    folder = 'fishnet_tiles'
)  

task.start()


In [ ]:
wa_basins_df = ee.data.computeFeatures({
    'expression': wa_basins,
    'fileFormat': 'PANDAS_DATAFRAME'
})